# 3. Evaluation pipeline

The initial results for the second model are quite mixed. Actually, they are quite bad in general. But it is a bit hard to understand why its performance is so bad without having a proper evaluation pipeline, that should use cross-validation as a way to mitigate the effects of the randomness of the train-test split.

In [18]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from skmultilearn.dataset import load_dataset


In [3]:
full_data = load_dataset("scene", "undivided")
train_data = load_dataset("scene", "train")
test_data = load_dataset("scene", "test")


scene:undivided - does not exists downloading
Downloaded scene-undivided
scene:train - exists, not redownloading
scene:test - exists, not redownloading


In [10]:
display(train_data[:2])
display(test_data[:2])
display(full_data[:2])


(<1211x294 sparse matrix of type '<class 'numpy.float64'>'
 	with 351805 stored elements in List of Lists format>,
 <1211x6 sparse matrix of type '<class 'numpy.int64'>'
 	with 1286 stored elements in List of Lists format>)

(<1196x294 sparse matrix of type '<class 'numpy.float64'>'
 	with 347724 stored elements in List of Lists format>,
 <1196x6 sparse matrix of type '<class 'numpy.int64'>'
 	with 1299 stored elements in List of Lists format>)

(<2407x294 sparse matrix of type '<class 'numpy.float64'>'
 	with 699529 stored elements in List of Lists format>,
 <2407x6 sparse matrix of type '<class 'numpy.int64'>'
 	with 2585 stored elements in List of Lists format>)

Just a side note here: this dataset is clearly too small. It might explain the wild results we were getting in the second model..

Anyway, let's keep going forward with the evaluation pipeline.

In [12]:
X_full, y_full, _, _ = full_data


In [21]:
dense_X_full = X_full.toarray()
dense_y_full = y_full.toarray()


skf = KFold(n_splits=5)
for i, (train_index, test_index) in enumerate(skf.split(dense_X_full, dense_y_full)):
    X_train = dense_X_full[train_index]
    y_train = dense_y_full[train_index]

    X_test = dense_X_full[test_index]
    y_test = dense_y_full[test_index]

    


Fold 0
TRAIN: 1925 TEST: 482
Fold 1
TRAIN: 1925 TEST: 482
Fold 2
TRAIN: 1926 TEST: 481
Fold 3
TRAIN: 1926 TEST: 481
Fold 4
TRAIN: 1926 TEST: 481


**Notice**: `StratifiedKFold` **cannot** be used as it does not support multilabel ([source](https://stackoverflow.com/questions/48508036/sklearn-stratifiedkfold-valueerror-supported-target-types-are-binary-mul)).

In [34]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from skmultilearn.problem_transform import ClassifierChain
from sklearn.svm import SVC
import sklearn.metrics as metrics


In [38]:
clf = ClassifierChain(
    classifier=SVC(),
    require_dense=[False, True]
)

s = {"accuracy": make_scorer(metrics.accuracy_score),
     "hamming_loss": make_scorer(metrics.hamming_loss, greater_is_better=False)}

r = cross_validate(clf, X_full, y_full, cv=5, scoring=s, return_train_score=True)


In [40]:
r

{'fit_time': array([6.38895631, 5.47674799, 3.91034126, 4.25431085, 3.89205217]),
 'score_time': array([0.90128756, 0.72894812, 0.67406583, 0.93108773, 0.71495819]),
 'test_accuracy': array([0.60788382, 0.62448133, 0.56340956, 0.75883576, 0.6049896 ]),
 'train_accuracy': array([0.7974026 , 0.79792208, 0.80425753, 0.78971963, 0.80685358]),
 'test_hamming_loss': array([-0.12275242, -0.11687414, -0.13513514, -0.06618157, -0.11884962]),
 'train_hamming_loss': array([-0.05809524, -0.05800866, -0.05304604, -0.06022845, -0.05209415])}

In [39]:
r["test_accuracy"].mean(), r["test_accuracy"].std()

(0.6319200144926287, 0.06657267886278025)

In [42]:
r["test_hamming_loss"].mean(), r["test_hamming_loss"].std()

(-0.11195857523658352, 0.023752236934311968)

In [43]:
clf2 = ClassifierChain(
    classifier=SVC(),
    require_dense=[False, True],
    order=[3, 0, 2, 5, 4, 1]
)

s2 = {"accuracy": make_scorer(metrics.accuracy_score),
     "hamming_loss": make_scorer(metrics.hamming_loss, greater_is_better=False)}

r2 = cross_validate(clf2, X_full, y_full, cv=5,
                   scoring=s2, return_train_score=True)


In [44]:
r2

{'fit_time': array([4.16601849, 4.84276986, 4.13891101, 4.2367835 , 3.85717344]),
 'score_time': array([0.69529414, 0.74412417, 0.69608188, 0.70389557, 0.64631033]),
 'test_accuracy': array([0.1473029 , 0.39626556, 0.05613306, 0.28274428, 0.3035343 ]),
 'train_accuracy': array([0.31324675, 0.20727273, 0.3556594 , 0.26272066, 0.27050883]),
 'test_hamming_loss': array([-0.28284924, -0.20020747, -0.31185031, -0.23354123, -0.22903673]),
 'train_hamming_loss': array([-0.22995671, -0.26554113, -0.21581862, -0.25034614, -0.24575978])}

Ok, the initial results here show that the proposed Classifier Chain is indeed consistently bad. Regardless, now I have a better ideia of how to set up this evaluation pipeline, which I should conclude tomorrow.